#### 1. Import pandas library

In [1]:
import pandas as pd
import numpy as np

#### 2. Load data (users.csv, posts.csv)

In [2]:
users_df = pd.read_csv('users.csv')
posts_df = pd.read_csv('posts.csv')

In [3]:
users_df.head()

,Unnamed: 0,id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,0,107658,ivordesign,NaN,NaN,2009-05-15 12:21:51.230000+00:00,2014-08-28 17:43:18.040000+00:00,United Kingdom,305,2,0,91,NaN,http://www.ivorthedesigner.co.uk
1,1,218597,icabod,NaN,NaN,2009-11-25 13:30:42.460000+00:00,2020-05-29 11:44:18.140000+00:00,United Kingdom,6559,361,25,374,NaN,http://icablog.org/
2,2,326360,rbaker86,<p>Software developer and technology enthusias...,NaN,2010-04-26 21:18:19.857000+00:00,2017-12-04 16:49:29.297000+00:00,United Kingdom,1802,40,14,97,NaN,http://na
3,3,379556,Rui Marques,<p>Engineer and Automation Architect at Vonage...,NaN,2010-06-29 23:32:49.870000+00:00,2020-05-28 14:49:23.427000+00:00,United Kingdom,2335,125,10,261,NaN,http://ruimarques.io
4,4,450456,Jules,NaN,NaN,2010-09-17 09:47:09.397000+00:00,2020-05-13 07:03:58.113000+00:00,United Kingdom,7746,256,50,3598,https://i.stack.imgur.com/4Y8Zf.jpg?s=128&g=1,https://www.julesmoorhouse.com


In [4]:
posts_df.head()

,id,owner_user_id,score,view_count,comment_count,favorite_count
0,30336926,NaN,1,14,3,NaN
1,36873524,NaN,1,18,0,1.0
2,36605876,NaN,1,19,0,NaN
3,36718461,NaN,1,17,0,NaN
4,30434893,NaN,0,15,0,NaN


#### 3. Rename id column to user_id

In [5]:
users_df.rename(columns={'id':'user_id'}, inplace=True)

In [6]:
users_df.head(5)

,Unnamed: 0,user_id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,0,107658,ivordesign,NaN,NaN,2009-05-15 12:21:51.230000+00:00,2014-08-28 17:43:18.040000+00:00,United Kingdom,305,2,0,91,NaN,http://www.ivorthedesigner.co.uk
1,1,218597,icabod,NaN,NaN,2009-11-25 13:30:42.460000+00:00,2020-05-29 11:44:18.140000+00:00,United Kingdom,6559,361,25,374,NaN,http://icablog.org/
2,2,326360,rbaker86,<p>Software developer and technology enthusias...,NaN,2010-04-26 21:18:19.857000+00:00,2017-12-04 16:49:29.297000+00:00,United Kingdom,1802,40,14,97,NaN,http://na
3,3,379556,Rui Marques,<p>Engineer and Automation Architect at Vonage...,NaN,2010-06-29 23:32:49.870000+00:00,2020-05-28 14:49:23.427000+00:00,United Kingdom,2335,125,10,261,NaN,http://ruimarques.io
4,4,450456,Jules,NaN,NaN,2010-09-17 09:47:09.397000+00:00,2020-05-13 07:03:58.113000+00:00,United Kingdom,7746,256,50,3598,https://i.stack.imgur.com/4Y8Zf.jpg?s=128&g=1,https://www.julesmoorhouse.com


#### 4. Rename id column to post_id and owner_user_id to user_id

In [7]:
posts_df.rename(columns={'id':'post_id', 'owner_user_id':'user_id'}, inplace=True)

In [8]:
len(posts_df['user_id'].unique())

223872

#### 5. Define new dataframes for users and posts with the following selected columns:
    **users columns**: user_id, reputation,views,up_votes,down_votes
    **posts columns**: post_id, score,user_id,view_count,comment_count

In [9]:
new_users_df = users_df.loc[:,['user_id','reputation','views','up_votes','down_votes']].copy()

In [10]:
new_posts_df = posts_df.loc[:,['post_id','score','user_id','view_count','comment_count']].copy()

#### 6. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [11]:
merged_df = new_posts_df.merge(new_users_df, how='outer', left_on='user_id', right_on='user_id').copy()

In [12]:
# with outer merge we get all the rows from both dataframes
# even if they don't match user_id with each other
merged_df[merged_df['user_id'].isna() != True]

,post_id,score,user_id,view_count,comment_count,reputation,views,up_votes,down_votes
1242,28532353.0,1.0,4453293.0,18.0,0.0,NaN,NaN,NaN,NaN
1243,37342127.0,0.0,1876983.0,9.0,0.0,NaN,NaN,NaN,NaN
1244,36829997.0,1.0,1876983.0,16.0,1.0,NaN,NaN,NaN,NaN
1245,34535507.0,0.0,1876983.0,15.0,0.0,NaN,NaN,NaN,NaN
1246,34961481.0,0.0,1876983.0,18.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
339789,NaN,NaN,574332.0,NaN,NaN,41.0,6.0,10.0,0.0
339790,NaN,NaN,7155452.0,NaN,NaN,41.0,6.0,0.0,0.0
339791,NaN,NaN,5529001.0,NaN,NaN,41.0,11.0,0.0,0.0
339792,NaN,NaN,4105549.0,NaN,NaN,41.0,11.0,2.0,0.0


#### 7. How many missing values do you have in your merged dataframe? 

In [13]:
'''
By using outer merge we do not miss any rows from the original dataframes
'''

'\nBy using outer merge we do not miss any rows from the original dataframes\n'

In [14]:
# counting values NaN 
listcount = []
for i in merged_df.columns:
    listcount.append(merged_df[i].isna().sum())

In [15]:
print('Total number of NaN values (missing values) in the dataframe: ',sum(listcount))

Total number of NaN values (missing values) in the dataframe:  1356158


#### Bonus: Identify extreme values in your merged dataframe, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder. Hint: post_id cannot have outliers!

In [90]:
# We will use Tukey's rule approach to filter outliers
# we first check out the df with describe
merged_df_2.describe()

,post_id,score,user_id,view_count,comment_count,reputation,views,up_votes,down_votes
count,1.065000e+03,1065.000000,1.065000e+03,1065.000000,1065.000000,1065.000000,1065.000000,1065.000000,1065.000000
mean,3.398846e+07,0.123944,2.280169e+06,15.606573,0.926761,7251.176526,615.245070,397.688263,49.415023
std,4.495709e+06,0.578141,1.833959e+06,3.711159,1.627503,24021.612286,1497.407529,747.628862,177.308773
min,4.142174e+06,-7.000000,2.740000e+02,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,3.289111e+07,0.000000,6.532920e+05,13.000000,0.000000,246.000000,39.000000,15.000000,0.000000
50%,3.556317e+07,0.000000,1.996838e+06,17.000000,0.000000,1145.000000,145.000000,96.000000,3.000000
75%,3.699112e+07,0.000000,3.545704e+06,19.000000,1.000000,5206.000000,496.000000,375.000000,20.000000
max,3.776986e+07,3.000000,6.449207e+06,20.000000,11.000000,287170.000000,23103.000000,7886.000000,2692.000000


In [94]:
# we can see some extreme values on reputation, views, up_votes,down_votes
# we will filter outliers on those columns by applying Tukey's rule
# source: https://sksblogspace.wordpress.com/2017/05/05/outlier-elimination-tukeys-method/
# first we calculate the quartiles and interquartile
outliers_quartiles_reputation = np.percentile(merged_df_2['reputation'],[25,50,75])
Inter_reputation = outliers_quartiles_reputation[2] - outliers_quartiles_reputation[0]
outliers_quartiles_views = np.percentile(merged_df_2['views'],[25,50,75])
Inter_views = outliers_quartiles_views[2] - outliers_quartiles_views[0]
outliers_quartiles_upvotes = np.percentile(merged_df_2['up_votes'],[25,50,75])
Inter_upvotes = outliers_quartiles_upvotes[2] - outliers_quartiles_upvotes[0]
outliers_quartiles_downvotes = np.percentile(merged_df_2['down_votes'],[25,50,75])
Inter_downvotes = outliers_quartiles_downvotes[2] - outliers_quartiles_downvotes[0]

In [103]:
# We calculate the top and bottom thresholds applying Tukey's formula with 1.5
top_rep, bot_rep = (outliers_quartiles_reputation[2] + 1.5 * Inter_reputation,
                    outliers_quartiles_reputation[0] - 1.5 * Inter_reputation )
top_view, bot_view = (outliers_quartiles_views[2] + 1.5 * Inter_views,
                    outliers_quartiles_views[0] - 1.5 * Inter_views )
top_up, bot_up = (outliers_quartiles_upvotes[2] + 1.5 * Inter_upvotes,
                    outliers_quartiles_upvotes[0] - 1.5 * Inter_upvotes )
top_down, bot_down = (outliers_quartiles_downvotes[2] + 1.5 * Inter_downvotes,
                    outliers_quartiles_downvotes[0] - 1.5 * Inter_downvotes )

In [113]:
# we filter out the outliers using query
Non_outliers_df = merged_df_2.query('(reputation < @top_rep) & (reputation > @bot_rep ) & (views < @top_view) & (views > @bot_view) & (up_votes < @top_up) & (up_votes > @bot_up) & (down_votes < @top_down) & (down_votes > @bot_down)')


In [115]:
# as we can see extreme values are gone now (200 more or less rows have dissapeared)
Non_outliers_df.describe()

,post_id,score,user_id,view_count,comment_count,reputation,views,up_votes,down_votes
count,8.240000e+02,824.000000,8.240000e+02,824.000000,824.000000,824.000000,824.000000,824.000000,824.000000
mean,3.407986e+07,0.121359,2.584472e+06,15.567961,0.942961,1609.328883,163.700243,121.783981,5.910194
std,4.386367e+06,0.601602,1.809503e+06,3.755950,1.614906,2345.498907,207.379147,171.703478,9.842586
min,1.341497e+07,-7.000000,2.266000e+03,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,3.294887e+07,0.000000,1.124378e+06,13.000000,0.000000,127.000000,25.750000,8.000000,0.000000
50%,3.568497e+07,0.000000,2.335226e+06,17.000000,0.000000,701.000000,89.000000,46.000000,1.000000
75%,3.705268e+07,0.000000,3.782350e+06,19.000000,1.000000,1858.250000,215.000000,167.000000,7.250000
max,3.776986e+07,3.000000,6.449207e+06,20.000000,10.000000,11837.000000,1116.000000,867.000000,49.000000


In [116]:
# we generate the csv
Non_outliers_df.to_csv('Non_outliers_df.csv')

In [138]:
# if we want to have a dataframe with only the outliers
# we can check it this way
# we concat both dataframes (with and without outliers)
concat_df = pd.concat([merged_df_2,Non_outliers_df])

In [169]:
# if we drop all duplicates only remains that were present in one dataframe
# will remain. That would be the elements that were not present on Non_outliers_df
outliers_df = concat_df.drop_duplicates(keep=False)


In [171]:
# we create the csv with the outliers
outliers_df.to_csv('Outliers_df.csv')

In [174]:
# we check that the sum of len of Outliers + len of Non-Outliers = len of the complete dataframe
len(Non_outliers_df) + len(outliers_df) == len(merged_df_2)

True